# RTEM Dashboard
Author: SS  
Date: 10/05/2022  
Description: Notebook to explore the structure of the RTEM API and plot useful charts  

## Import libraries

In [28]:
# Standard python libraries
import pandas as pd
import json
import numpy as np

In [29]:
# RTEM libraries
from onboard.client import RtemClient
from onboard.client.dataframes import points_df_from_streaming_timeseries
from onboard.client.models import PointSelector, TimeseriesQuery, PointData
from datetime import datetime, timezone, timedelta
import pytz

## Load secret
This is done so we dont share API keys.   
To get this working on your notebook, create a folder called 'secrets' in your root directory and create a 'secrets.txt' inside it with your API key.

In [30]:
# Read secret from /secrets/ folder
with open('secrets/secrets.txt') as f:
    secret = f.readlines()[0]

# Setup client
client = RtemClient(api_key=secret)

In [31]:
# Run this once to minimise making too many calls
df_buildings = pd.DataFrame(client.get_all_buildings())

In [32]:
df_buildings.head()

,id,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,info,status,equip_count,point_count
0,441,5,88737,None,551143.0,None,None,None,None,America/New_York,"{'floors': '', 'm2fend': '', 'satend': '', 'su...",LIVE,20,1293
1,140,5,83483,None,21869.0,None,None,None,None,America/New_York,"{'floors': '', 'm2fend': '', 'satend': '', 'su...",LIVE,13,354
2,191,5,94414,None,198057.0,None,None,None,None,America/New_York,"{'floors': '', 'm2fend': '23:00', 'satend': '2...",LIVE,7,19
3,231,5,89496,None,NaN,None,None,None,None,America/New_York,"{'floors': '', 'm2fend': '', 'satend': '', 'su...",LIVE,4,51
4,248,5,116742,None,127000.0,None,None,None,None,America/New_York,"{'floors': '', 'm2fend': '', 'satend': '', 'su...",LIVE,7,34


In [49]:
print(f'shape of dataframe {df_buildings.shape}')
df_buildings.describe()

shape of dataframe (229, 14)


,id,org_id,sq_ft,equip_count,point_count
count,229.000000,229.0,2.010000e+02,229.000000,229.000000
mean,279.296943,5.0,3.598326e+05,26.572052,193.842795
std,126.752691,0.0,5.004452e+05,50.564882,510.273853
min,98.000000,5.0,7.960000e+03,1.000000,1.000000
25%,165.000000,5.0,9.000000e+04,4.000000,22.000000
50%,259.000000,5.0,1.980570e+05,8.000000,38.000000
75%,392.000000,5.0,3.640000e+05,25.000000,182.000000
max,503.000000,5.0,3.000000e+06,393.000000,6068.000000


### Something interesting going on in the info column

In [50]:
df_buildings['info'][0]

{'floors': '',
 'm2fend': '',
 'satend': '',
 'sunend': '',
 'geoCity': 'Bronx',
 'geoState': 'NY',
 'm2fstart': '',
 'satstart': '',
 'sunstart': '',
 'yearBuilt': '',
 'geoCountry': 'US',
 'weatherRef': '',
 'customerType': 'Multifamily'}

In [51]:
# splitting these values into columns in the df
objs = [df_buildings, pd.DataFrame(df_buildings['info'].tolist()).iloc[:, :14]]
df_buildings_flt = pd.concat(objs, axis=1).drop('info', axis=1)

# Plotting

In [52]:
df_buildings_flt.columns

Index(['id', 'org_id', 'name', 'address', 'sq_ft', 'image_src',
       'bms_manufacturer', 'bms_product_name', 'bms_version', 'timezone',
       'status', 'equip_count', 'point_count', 'floors', 'm2fend', 'satend',
       'sunend', 'geoCity', 'geoState', 'm2fstart', 'satstart', 'sunstart',
       'yearBuilt', 'geoCountry', 'weatherRef', 'customerType'],
      dtype='object')

In [53]:
# Import plottting libraries
import plotly
import seaborn as sns
pd.options.plotting.backend = "plotly" 

In [54]:
df_buildings_flt[['sq_ft']].plot(kind='hist',title = "Histogram of building area")

In [55]:
df_buildings_flt.plot(kind='scatter',
x = 'sq_ft',
y = 'customerType',
title = "Building Typology by area",
c='geoCity')

In [56]:
df_buildings_flt[["customerType"]].plot(kind='bar', title = "Counts of buildings by curtomer type")

## Now some function to help select buildings

In [41]:
def sb_buildings_by_function(df,function_string):
    """Function to get all buildings from the all buildings dataframe and filter them by function
    params:
    df: A pandas dataframe
    function_string : A list of string functions
    return: Filtered pandas dataframe"""
    filtered_df = df[df["customerType"]==function_string]
    return filtered_df
    

In [57]:
df_multifamily = sb_buildings_by_function(df_buildings_flt,"Multifamily")
df_multifamily.head()

,id,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,...,sunend,geoCity,geoState,m2fstart,satstart,sunstart,yearBuilt,geoCountry,weatherRef,customerType
0,441,5,88737,None,551143.0,None,None,None,None,America/New_York,...,,Bronx,NY,,,,,US,,Multifamily
4,248,5,116742,None,127000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Multifamily
6,434,5,129666,None,56090.0,None,None,None,None,America/New_York,...,,Brooklyn,NY,,,,,US,,Multifamily
11,317,5,128286,None,35000.0,None,None,None,None,America/New_York,...,,New York,NY,,,,,US,,Multifamily
13,375,5,97789,None,232808.0,None,None,None,None,America/New_York,...,,BRONX,NY,,,,,US,,Multifamily


## Going into the building sensors now

In [58]:
#Lets start with this building
all_buildings = df_multifamily["id"].to_list()

In [ ]:
'Hot Water Supply Temperature',
'Hot Water Return Temperature',
'Status',
'Hot Water Supply Temperature Setpoint',
'Enable'


In [90]:
query = PointSelector()
query.point_types     = ['Hot Water Supply Temperature']        # can list multiple point
query.equipment_types = ['boiler']     # types, equipment types,
query.buildings       = all_buildings        # buildings, etc.
selection = client.select_points(query)
points = selection["points"]

sensor_metadata = pd.DataFrame(client.get_points_by_ids(points))

#start_date = sensor_metadata.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min()
#end_date = sensor_metadata.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).max()

tz = pytz.timezone('UTC')
start_date = datetime(2018,1,1,0,0,0).replace(tzinfo=tz)
end_date = datetime(2018,12,31,0,0,0).replace(tzinfo=tz)

timeseries_query = TimeseriesQuery(point_ids = selection['points'], start = start_date, end = end_date)
sensor_data = points_df_from_streaming_timeseries(client.stream_point_timeseries(timeseries_query))

In [92]:
df_sensor = sensor_metadata[["building_id","id","name","description","value"]]
#df_sensor = df_sensor.description.fillna(value=np.nan, inplace=True)
df_sensor['description'] = df_sensor['description'].fillna(df_sensor['name'])
df_sensor = df_sensor.drop(columns="name")
df_sensor["value"] = pd.to_numeric(df_sensor["value"])
df_sensor["short_name"] = df_sensor['description'].str[:15]
df_sensor.head()


C:\Users\ssanjay\AppData\Local\Temp\ipykernel_5048\3646713211.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,building_id,id,description,value,short_name
0,111,194188,237 W 100th St Boiler Domestic Hot Water Tempe...,173.35,237 W 100th St
1,111,194193,237 W 100th St Boiler Hot Water Coil Temperature,177.47,237 W 100th St
2,127,197306,Boiler1 Temp,187.00,Boiler1 Temp
3,127,197308,Boiler2 Temp,184.25,Boiler2 Temp
4,128,197392,800 Grand Concourse Boiler Boiler 1 Domestic P...,190.94,800 Grand Conco


In [93]:
df_sensor.plot.bar(y = "description",
 x="value",
 barmode = "relative",
 color='value',
 hover_data=["value","description"],
 height = 800,
 title = "Hot Water Supply Temperature")

In [62]:
df_sensor.plot.bar(y = "description",
 x="value",
 barmode = "relative",
 color='value',
 hover_data=["value","description"],
 height = 800,
 title = "Hot Water Return Temperature")

## Extending the functionality to multiple sensors

### Step 1 - Get all the buildings

In [ ]:
# First fetch all the buildings
df_buildings = pd.DataFrame(client.get_all_buildings())
# Flattern the "info" column into multiple columns in the df
objs = [df_buildings, pd.DataFrame(df_buildings['info'].tolist()).iloc[:, :14]]
df_buildings_flt = pd.concat(objs, axis=1).drop('info', axis=1)

In [ ]:
def get_building_info(building_id,df_buildings_flt):
    """_summary_

    Args:
        building_id (_type_): _description_
        df_buildings_flt (_type_): _description_

    Returns:
        _type_: _description_
    """
    building_info = df_buildings_flt[df_buildings_flt["id"]==building_id].to_dict()
    return building_info

In [158]:
all_equipment = pd.DataFrame(client.get_building_equipment(441))[['id', 'building_id', 'equip_id',  'points', 'tags']]

In [160]:
all_equipment.head()

,id,building_id,equip_id,points,tags
0,28797,441,boiler-1,"[{'id': 310029, 'building_id': 441, 'last_upda...","[boiler, hvac]"
1,28798,441,boiler-2,"[{'id': 310079, 'building_id': 441, 'last_upda...","[boiler, hvac]"
2,28799,441,boiler-3,"[{'id': 310108, 'building_id': 441, 'last_upda...","[boiler, hvac]"
3,28800,441,boiler-4,"[{'id': 310144, 'building_id': 441, 'last_upda...","[boiler, hvac]"
4,28801,441,boiler-5,"[{'id': 310152, 'building_id': 441, 'last_upda...","[boiler, hvac]"


In [182]:
csv_eq_types = pd.read_csv('data/equip_types.csv')
csv_point_types = pd.read_csv('data/point_types.csv')

['Hot Water Supply Temperature',
 'Hot Water Supply Temperature Setpoint',
 'Hot Water Return Temperature',
 'Enable',
 'Status']

In [203]:
building_id = 441
eq_types = ['boiler']
query = PointSelector()
query.point_types = ['Hot Water Supply Temperature']
query.equipment_types = [eq_types[0]]
query.buildings = [building_id]
selection = client.select_points(query)
selection


{'orgs': [5],
 'buildings': [441],
 'equipment': [28800, 28797, 28798, 28799],
 'equipment_types': [19],
 'point_types': [88],
 'points': [310025, 310061, 310096, 310134, 310044]}

Now that we have an example to see what the different values are, lets make a function around it to query something about one building

In [450]:
def doctor_my_building(building_id,df_buildings_flt,year):
    """_summary_

    Args:
        building_id (_type_): _description_
        df_buildings_flt (_type_): _description_
        year (_type_): _description_

    Returns:
        _type_: _description_
    """
    dflist_equipments = []
    dflist_metadata = []
    from matplotlib import pyplot as plt
    building_info = get_building_info(building_id,df_buildings_flt)
    id = list(building_info["id"].values())[0]
    building_area = list(building_info["sq_ft"].values())[0]
    building_city = list(building_info["geoCity"].values())[0]
    building_use = list(building_info["customerType"].values())[0]
    print(f"You have selected building {id}\nBuilding {id} is located in the {building_city} area of NY and has an area of {building_area} square feet.\nBuilding {building_id} belongs to the category {building_use}.")

    #Select the category
    df_buildingUse = sb_buildings_by_function(df_buildings_flt,building_use)
    # Lets start with this building
    all_buildings = df_buildingUse["id"].unique().tolist()
    print(f'There are {len(all_buildings)} other buildings available in this category')
    
    
    # Going into equipments
    all_equipment = pd.DataFrame(client.get_building_equipment(id))
    eq_types = all_equipment["equip_type_tag"].unique().tolist()
    print(f"The following equipment types are available {eq_types}")
    
    # Maybe this helps somewhere?
    #null_equipment = csv_eq_types[csv_eq_types["critical_point_types"] == '[]']["tag_name"].to_list()
    tz = pytz.timezone('UTC')
    start_date = datetime(year,1,1,0,0,0).replace(tzinfo=tz)
    end_date = datetime(year,12,31,0,0,0).replace(tzinfo=tz)
    # For each equipment type in this building
    for i in eq_types:
        print(f"    Fetching sensors for {i}...")
        # Look for ValueError: invalid literal for int() with base 10: '' 
        # and add the equipment that caused this error in the list below
        if i not in ['site','virtual','meter','panel','lighting','elevator','battery']:
            point_types = []
            for j in [int(x) for x in csv_eq_types[csv_eq_types['tag_name']==i]["critical_point_types"].to_list()[0].replace('[','').replace(']','').replace(' ','').split(',')]:
                point_type = csv_point_types[csv_point_types['id'] == j]["tag_name"].to_list()[0]
                point_types.append(point_type)
            eq_types = [i]
            query = PointSelector()
            query.point_types = point_types
            query.equipment_types = [i]
            query.buildings = [id]
            selection = client.select_points(query)
            print(f"        Following sensors are found: {point_types}")
            pd.options.plotting.backend = "plotly" 
            if len(selection['points'])>0:
                timeseries_query = TimeseriesQuery(point_ids = selection['points'], start = start_date, end = end_date)
                points = selection["points"]
                sensor_metadata = pd.DataFrame(client.get_points_by_ids(points))
                dflist_metadata.append(sensor_metadata)
                sensor_data = points_df_from_streaming_timeseries(client.stream_point_timeseries(timeseries_query))
                if "timestamp" in sensor_data.columns:
                    sensor_data =  sensor_data.set_index("timestamp")
                    print(f"            Appending sensor data for {i}")
                    dflist_equipments.append(sensor_data)
                else:
                    print(f"¯\_(ツ)_/¯ Ahhh.... I didn't find any timeseries data for {i}")
            else:
                print(f"¯\_(ツ)_/¯ Ahhh.... I didn't find any data for {i}")
        else:
            print("¯\_(ツ)_/¯ Ahh.... No data found")
    return dflist_equipments,dflist_metadata

def rename_sensors(sensors,metadata):
    """_summary_

    Args:
        sensors (_type_): _description_
        metadata (_type_): _description_
    """
    new_sensors = []
    id_list = []
    description_list = []
    for df in metadata:
        sublist_id = df["id"].to_list()
        for item in sublist_id:
            id_list.append(item)
        sublist_description = df["description"].to_list()
        for item in sublist_description:
            description_list.append(item)
    for df in sensors:
        key = df.columns.to_list()
        val = []
        for col in key:
            if col in id_list:
                index = id_list.index(col)
                val.append(description_list[index])
        column_rename = {key[i]: val[i] for i in range(len(key))}
        df = df.rename(column_rename,axis='columns')
        new_sensors.append(df)
    return(new_sensors)

## Testing out the function above

In [453]:
# Try 394 (1min51sec) - 6 sensors
# Try 111 (0min2.8sec) - 1 sensor
# Try 119 (0min7.5sec) - 3 sensors
# Try 169 (0min3.7sec) - 3 sensors - Checkout 2018 sensor[0].plot() here! 
sensors,metadata = doctor_my_building(169,df_buildings_flt,2018)
print(f"There are {len(sensors)} equipment(s) in the dataframe")
sensors = rename_sensors(sensors, metadata)

You have selected building 169
Building 169 is located in the Brooklyn area of NY and has an area of 552860.0 square feet.
Building 169 belongs to the category Multifamily.
There are 69 other buildings available in this category
The following equipment types are available ['site', 'virtual', 'battery', 'boiler', 'hotWaterPlant', 'pump']
    Fetching sensors for site...
¯\_(ツ)_/¯ Ahh.... No data found
    Fetching sensors for virtual...
¯\_(ツ)_/¯ Ahh.... No data found
    Fetching sensors for battery...
¯\_(ツ)_/¯ Ahh.... No data found
    Fetching sensors for boiler...
        Following sensors are found: ['Hot Water Supply Temperature', 'Hot Water Supply Temperature Setpoint', 'Hot Water Return Temperature', 'Enable', 'Status']
            Appending sensor data for boiler
    Fetching sensors for hotWaterPlant...
        Following sensors are found: ['Hot Water Supply Temperature', 'Hot Water Supply Temperature Setpoint', 'Hot Water Return Temperature', 'Hot Water Supply Flow', 'Hot Wa

In [454]:
sensors[0].plot()